<p>
    Justin Bucsa submission. 
</p>
    

<h1>Introduction to the problem sets

<p>
    
    Hi there,

    Thanks for taking an interest in this coding test. There are 7 problem sets:
   
    1) Calculate futures PNL
    2) Run custom summary stats
    3) Plot percentage returns using bokeh
    4) Gather and show economic events
    5) Load a JSON and create a loop to update settings
    6) Create an email function to send attachments (no need to include your password)
    7) IQfeed symbol lookup (no coding)
    
    Please complete all 7 tests using python in this ipython/jupyter notebook. 
    We will compare your results with an internal notebook.

    Please unzip the "mn_test" directory that was sent to you.
    Work in this directory to run the ipython/jupyter notebook titled "mn_test.ipynb".

    Please enter your name info in the code block below.
    
    Thanks and good luck!
</p>

In [1]:
# Example:  PERSON = "John Smith"
PERSON = "John Smith"

In [2]:
PERSON = PERSON.lower().replace(' ', "")
PERSON

'johnsmith'

<h1>1) Calculate futures PNL

If you don't know how to calculate futures pnl, you can use this resource to learn how:
https://www.cmegroup.com/education/courses/introduction-to-futures/calculating-futures-contract-profit-or-loss.html#
    
If you need to know where to find futures contract specs, search the given products "contract spec" webpage on the CME website (we're only using CME products in this example)

For each product in the /data/futures_data.csv file, read it in with pandas and calculate the PNL for each product.

Assume you place a "buy" long trade at the very first price for 5 contracts and hold till the very last price.

(don't worry about rolling futures or weekends, just act as if it's one continuous price data set)

Plot the PNL per product in it's own plot using bokeh.

What is the final PNL per product?

In [57]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import DatetimeTickFormatter

# Set output to display in Jupyter Notebook environment
output_notebook()

# Load the data
file_path = '../../Monick-Assessment/Dev_Test/data/futures_data.csv'

futures_data = pd.read_csv(file_path)

# Ensure timestamps are in datetime format
futures_data['timestamp'] = pd.to_datetime(futures_data['timestamp'])

# Define contract sizes for each product (values are examples, check CME specs for accuracy)
contract_sizes = {
    'YM': 5,  # Dow Jones E-mini Futures
    'ES': 50,  # S&P 500 E-mini Futures
    'NQ': 20,  # NASDAQ-100 E-mini Futures
    'RTY': 50,  # Russell 2000 E-mini Futures
}

# Initialize a dictionary to store final PNL per product
final_pnls = {}
plots = []

# Group by product symbol
grouped = futures_data.groupby('symbol')

for symbol, group in grouped:
    # Sort data by timestamp to ensure correct ordering
    group = group.sort_values('timestamp')
    
    # Get the initial and final prices
    initial_price = group['open'].iloc[0]
    final_price = group['close'].iloc[-1]
    
    # Get the contract size (default to 1 if not found)
    contract_size = contract_sizes.get(symbol, 1)
    
    # Calculate PNL
    num_contracts = 5
    pnl = (final_price - initial_price) * contract_size * num_contracts
    final_pnls[symbol] = pnl
    
    # Create a plot for this product
    p = figure(title=f"PNL for {symbol}", x_axis_label='Timestamp', y_axis_label='PNL ($)', 
               x_axis_type='datetime', width=500, height=300)
    
    # Plot PNL over time
    pnl_series = (group['close'] - initial_price) * contract_size * num_contracts
    p.line(group['timestamp'], pnl_series, legend_label=f"Final PNL: {pnl:.2f}", line_width=2)
    p.legend.location = "top_left"
    p.xaxis.formatter = DatetimeTickFormatter(days="%Y-%m-%d", hours="%H:%M")
    plots.append(p)

# Display all plots in a single column layout for simplicity
layout = column(*plots)
show(layout)

# Display final PNL per product
print("Final PNL per product:")
print(final_pnls)





Loading BokehJS ...

Final PNL per product:
{'ES': np.float64(531367.4999999999), 'NQ': np.float64(962617.0), 'RTY': np.float64(257905.00000000003), 'YM': np.float64(314222.99999999994)}


<h1>2) Run custom summary stats

We have a summary statistics format that we like to use for daily PNL analysis. 

First, convert the hourly futures data into daily PNLs. 

Then use the following summary_stats() function to produce the analysis.

Run two groups of summary stats: one for ES and another for NQ. 

If your column headers don't match what's in the summary_stats() function, please change them to match.

Use the Ipython display() function to show the final summary_stats dataframe.

In [59]:
import pandas as pd
import numpy as np

def summary_stats(df_: pd.DataFrame, column_name="value"):
    
    df_ = df_.dropna(how='all')
    # Changes "size" to "num_contracts"
    # df_["count"] = int(df_["size"].count())
    df_["count"] = int(df_["num_contracts"].count())
    df_["mean"] = round(df_.pnl.mean())
    df_["min"] = round(df_.pnl.min())
    df_["25(%)"] = round(df_.pnl.quantile(q=0.25))
    df_["50(%)"] = round(df_.pnl.quantile(q=0.5))
    df_["75(%)"] = round(df_.pnl.quantile(q=0.75))
    df_["max"] = round(df_.pnl.max())
    df_["wins"] = df_["pnl"].apply(lambda x: 1 if x > 0 else 0)
    df_["win(%)"] = round(int(len(df_[df_.wins > 0])/df_.shape[0] * 100), 2)
    df_["mean_unit_pnl"] = round(np.sum(df_.pnl)/ np.sum(np.absolute(df_["num_contracts"])))
    try:
        df_["std_unit_pnl"] = round(np.std(df_.pnl) / np.sum(np.absolute(df_["num_contracts"])))
    except:
        df_["std_unit_pnl"] = np.nan
    df_['average_daily_pnl'] = round(df_.groupby('date').pnl.sum().mean())
    df_['total_dates'] = round(len(df_.date.unique()))
    try:
        df_['std_daily_pnl'] = round(df_.groupby('date').pnl.sum().std())
    except:
        df_['std_daily_pnl'] = np.nan
    try:
        sharpe = np.round((252**0.5)*(df_["average_daily_pnl"] / df_["std_daily_pnl"]), 3)
    except:
        sharpe = np.nan
    df_["sharpe"] = round(sharpe, 2)
    df_["total_pnl"] = round(np.sum(df_.pnl))
    df_["total_volume"] = round(np.sum(np.absolute(df_["num_contracts"])))
    df_final = df_.iloc[0][['count', 'mean', 'min', '25(%)',
                            '50(%)', '75(%)', 'max', 'win(%)', 'mean_unit_pnl', 'std_unit_pnl',
                            'average_daily_pnl', 'total_dates', 'std_daily_pnl', 'sharpe',
                            'total_pnl', 'total_volume']].reset_index()
    df_final.columns = ['statistic', column_name]
    df_final = df_final.set_index('statistic')
    return df_final


In [61]:
# Convert hourly data to daily PNLs
futures_data['date'] = futures_data['timestamp'].dt.date
futures_data['pnl'] = (futures_data['close'] - futures_data['open']) * futures_data['symbol'].map(contract_sizes) * 5
futures_data['num_contracts'] = 5  # Assuming 5 contracts per trade
futures_data['daily_return'] = (futures_data['close'] - futures_data['open']) / futures_data['open']
daily_pnl = futures_data.groupby(['symbol', 'date'])[['pnl', 'num_contracts', 'daily_return']].sum().reset_index()

# Run summary stats for ES and NQ
daily_es = daily_pnl[daily_pnl['symbol'] == 'ES']
daily_nq = daily_pnl[daily_pnl['symbol'] == 'NQ']

es_stats = summary_stats(daily_es, column_name="ES")
nq_stats = summary_stats(daily_nq, column_name="NQ")

from IPython.display import display
print("Summary stats for ES:")
display(es_stats)
print("\nSummary stats for NQ:")
display(nq_stats)




Summary stats for ES:


,ES
statistic,
count,1046
mean,-34
min,-21000
25(%),-208
50(%),0
75(%),203
max,19500
win(%),48
mean_unit_pnl,0



Summary stats for NQ:


,NQ
statistic,
count,1046
mean,-182
min,-30000
25(%),-330
50(%),39
75(%),392
max,18600
win(%),54
mean_unit_pnl,-2


<h1>3) Plot percentage returns using bokeh

You should reuse the code from 1 and 2 to build your datasets and make you plots.

Make a dual line plot of daily % returns per product.

In [56]:



# Plot percentage returns using Bokeh
p_return = figure(title="Daily Percentage Returns", x_axis_label="Date", y_axis_label="% Return", x_axis_type="datetime", width=700, height=400)

# Add ES and NQ percentage returns
p_return.line(daily_es['date'], daily_es['daily_return'] * 100, legend_label="ES", line_width=2, color="blue")
p_return.line(daily_nq['date'], daily_nq['daily_return'] * 100, legend_label="NQ", line_width=2, color="green")

p_return.legend.location = "top_left"
p_return.xaxis.formatter = DatetimeTickFormatter(days="%Y-%m-%d")

# Show plot
show(p_return)

# Compare volatility
es_volatility = daily_es['daily_return'].std() * np.sqrt(252) * 100  # Annualized volatility in %
nq_volatility = daily_nq['daily_return'].std() * np.sqrt(252) * 100  # Annualized volatility in %

print(f"Annualized Volatility for ES: {es_volatility:.2f}%")
print(f"Annualized Volatility for NQ: {nq_volatility:.2f}%")







Annualized Volatility for ES: 4.83%
Annualized Volatility for NQ: 4.97%


<h1>4) Gather and show economic events

Please gather the week's economic events below and sort by earliest date, time.

You will have to use python to scrape a website to do this. 

Use the chromedriver we provided in "driver" folder and use selenium for this task. 

You will have to use a chrome web browser and you may get a chromedriver error. 

Troubleshooting this potential error is a part of the task.

In [7]:
events_url = 'https://www.forexfactory.com/calendar'

In [8]:
# you may find this helpful
import os
EXECUTABLE_PATH = os.path.join(os.getcwd(), "driver", "chromedriver")

<p>
    Below are a few examples to solving this problem. They all have a similar issue and cannot find the exact HTML element to scrape from. This was a great example of showing a website that has nested elements in them. Due to the level of nesting, selenium is having difficult locating the data it wants to grab from the specific element. 
</p>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import os
EXECUTABLE_PATH = os.path.join(os.getcwd(), "driver", "chromedriver")

# Path to your chromedriver (adjust as needed)
CHROMEDRIVER_PATH = '../../Monick-Assessment/Dev_Test/driver/chromedriver.exe'

def get_economic_events(events_url):
    service = Service(CHROMEDRIVER_PATH)
    options = webdriver.ChromeOptions()
    # Uncomment to see the browser for debugging
    # options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    try:
        print("Navigating to the page...")
        driver.get(events_url)

        print("Waiting for the calendar to load...")
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'calendar__table'))
        )

        rows = driver.find_elements(By.CSS_SELECTOR, 'calendar__header--desktop subhead')
        print(f"Found {len(rows)} rows in the calendar.")

        events = []
        for row in rows:
            try:
                # Update these selectors based on your inspection of the website
                date = row.find_element(By.CSS_SELECTOR, 'calendar__row calendar__row--day-breaker').text
                time = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__time').text
                currency = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__currency').text
                impact = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__impact').get_attribute('class').split()[-1]  # Extracts impact level
                event_name = row.find_element(By.CSS_SELECTOR, 'calendar__event-title').text
                actual = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__actual').text
                forecast = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__forecast').text
                previous = row.find_element(By.CSS_SELECTOR, 'calendar__cell calendar__previous').text

                print(f"Row data: Date={date}, Time={time}, Currency={currency}, Impact={impact}, Event={event_name}")

                events.append({
                    'Date': date,
                    'Time': time,
                    'Currency': currency,
                    'Impact': impact,
                    'Event': event_name,
                    'Actual': actual,
                    'Forecast': forecast,
                    'Previous': previous
                })
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        print("Scraped events:", events)

        events_df = pd.DataFrame(events)
        if not events_df.empty:
            events_df['Datetime'] = pd.to_datetime(events_df['Date'] + ' ' + events_df['Time'], errors='coerce')
            events_df = events_df.dropna(subset=['Datetime'])
            events_df = events_df.sort_values('Datetime')
            events_df = events_df.drop(columns=['Datetime'])

        return events_df

    finally:
        driver.quit()

events_url = 'https://www.forexfactory.com/calendar'
economic_events_df = get_economic_events(events_url)
print(economic_events_df)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Configure Selenium WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Path to ChromeDriver (update the path as needed)
chrome_driver_path = '../../Monick-Assessment/Dev_Test/driver/chromedriver.exe'
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://www.forexfactory.com/calendar"

# Load the webpage
driver.get(url)

# Wait for the calendar to load completely
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "tr.calendar__row"))
    )
    print("Calendar table loaded successfully.")
except Exception as e:
    print(f"Error loading calendar table: {e}")
    driver.quit()
    exit()

# Extract data from the calendar
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "tr.calendar__row")

print(f"Number of rows found: {len(rows)}")  # Debugging line

for row in rows:
    try:
        time_cell = row.find_element(By.CLASS_NAME, "calendar__time")
        currency_cell = row.find_element(By.CLASS_NAME, "calendar__currency")
        event_cell = row.find_element(By.CLASS_NAME, "calendar__event")
        actual_cell = row.find_element(By.CLASS_NAME, "calendar__actual")
        forecast_cell = row.find_element(By.CLASS_NAME, "calendar__forecast")
        previous_cell = row.find_element(By.CLASS_NAME, "calendar__previous")

        # Debugging lines to log the extracted data
        print(f"Time: {time_cell.text.strip()}, Currency: {currency_cell.text.strip()}, Event: {event_cell.text.strip()}")

        data.append({
            "Time": time_cell.text.strip(),
            "Currency": currency_cell.text.strip(),
            "Event": event_cell.text.strip(),
            "Actual": actual_cell.text.strip(),
            "Forecast": forecast_cell.text.strip(),
            "Previous": previous_cell.text.strip(),
        })
    except Exception as e:
        print(f"Error processing row: {e}")

# Close the WebDriver
driver.quit()

# Convert the data into a DataFrame
calendar_df = pd.DataFrame(data)

# Check if DataFrame is empty (debugging)
if calendar_df.empty:
    print("No data was scraped.")
else:
    print(calendar_df.head())

# Save the data to a CSV file
csv_filename = "forex_calendar_data.csv"
calendar_df.to_csv(csv_filename, index=False)

print(f"Data has been successfully scraped and saved to {csv_filename}.")


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time 

import os


# Path to your chromedriver (adjust as needed)
CHROMEDRIVER_PATH = '../../Monick-Assessment/Dev_Test/driver/chromedriver.exe' 

def get_economic_events(events_url):
    service = Service(CHROMEDRIVER_PATH)
    options = webdriver.ChromeOptions()
    # Uncomment to see the browser for debugging
    # options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    try:
        print("Navigating to the page...")
        driver.get(events_url)

        print("Waiting for the calendar to load...")
        # Wait for the calendar table to be present
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'table.calendar__table')) 
        )

        # Wait for the rows to be present (adjust the selector if needed)
        WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tr.calendar__row'))
        )
        rows = driver.find_elements(By.CSS_SELECTOR, 'tr.calendar__row')  
        print(f"Found {len(rows)} rows in the calendar.")

        events = []
        for row in rows:
            try:
                # Update these selectors based on your inspection of the website
                date = row.find_element(By.CSS_SELECTOR, 'div.calendar__date').text.split('\n')[0]
                time = row.find_element(By.CSS_SELECTOR, 'td.calendar__time').text
                currency = row.find_element(By.CSS_SELECTOR, 'td.calendar__currency').text
                impact = row.find_element(By.CSS_SELECTOR, 'td.calendar__impact span').get_attribute('class').split()[-1]
                event_name = row.find_element(By.CSS_SELECTOR, 'td.calendar__event span').text
                actual = row.find_element(By.CSS_SELECTOR, 'td.calendar__actual').text
                forecast = row.find_element(By.CSS_SELECTOR, 'td.calendar__forecast').text
                previous = row.find_element(By.CSS_SELECTOR, 'td.calendar__previous').text

                print(f"Row data: Date={date}, Time={time}, Currency={currency}, Impact={impact}, Event={event_name}")

                events.append({
                    'Date': date,
                    'Time': time,
                    'Currency': currency,
                    'Impact': impact,
                    'Event': event_name,
                    'Actual': actual,
                    'Forecast': forecast,
                    'Previous': previous
                })
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        print("Scraped events:", events)

        events_df = pd.DataFrame(events)
        if not events_df.empty:
            events_df['Datetime'] = pd.to_datetime(events_df['Date'] + ' ' + events_df['Time'], errors='coerce')
            events_df = events_df.dropna(subset=['Datetime'])
            events_df = events_df.sort_values('Datetime')
            events_df = events_df.drop(columns=['Datetime'])

        return events_df

    finally:
        driver.quit()

events_url = 'https://www.forexfactory.com/calendar'
economic_events_df = get_economic_events(events_url)
print(economic_events_df)

<h1>5) Load a JSON and create a loop to update settings

Find the JSON file in /data/RetailSales_CHI.json.

Load this file here and create a function to loop over the "params" and update the following settings:
    
- Change "event_date" to "2021-12-14"
- Change "event_time" to "09:45:00"
- Use event_time variable with a python/pandas timedelta builtin function to add "5 minutes" and save this as the "hedge_start_time"

In [65]:
import json
from datetime import timedelta
import pandas as pd

# Load the RetailSales_CHI.json file
file_path = '../../Monick-Assessment/Dev_Test/data/RetailSales_CHI.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to update settings in the params section
def update_settings(data, event_date="2021-12-14", event_time="09:45:00"):
    for key, settings in data['params'].items():
        settings['event_date'] = event_date
        settings['event_time'] = event_time
        
        # Calculate hedge_start_time by adding 5 minutes to event_time
        event_time_obj = pd.to_datetime(event_time)
        hedge_start_time = (event_time_obj + timedelta(minutes=5)).time().strftime('%H:%M:%S')
        settings['hedge_start_time'] = hedge_start_time
    
    return data

# Update the settings
updated_data = update_settings(data)

# Display the updated JSON structure
print(json.dumps(updated_data, indent=4))


{
    "locale": "Nevada",
    "city": "Fallon",
    "street": "RetailSales_CHI",
    "tag": "BF",
    "trading_group": "general",
    "fcm_id": "Advantage",
    "ib_id": "Monick",
    "level": "PROD_ADV",
    "account_id": "737C9261",
    "real_time_update": false,
    "is_active": true,
    "params": {
        "NKD1": {
            "quote_interval": 4,
            "aliases_to_listen": [
                "NKD1"
            ],
            "max_position": 1,
            "contract_size": 1,
            "max_not_complete_orders_per_side": 1,
            "market_modes": [
                "Open"
            ],
            "stop_profit_in_ticks": 500,
            "stop_loss_in_ticks": 20,
            "stop_out_ticks": 5,
            "max_orders_per_price_level": 1,
            "round_out_per_side": true,
            "event_date": "2021-12-14",
            "event_time": "09:45:00",
            "hedge_start_time": "09:50:00",
            "cancel_time_post_event_in_seconds": 10,
            "stra

<h1>6) Create an email function to send attachments

You will likely have to consider additional python packages to send an email.

You don't have to include any personal information such as email address and password. 

Just test something that does in fact send an email. 

In [10]:
# you could email an attachment with this naming but not necessary, just show that the email function works
ZIP_DIR_NAME = f"{PERSON}_mn_test"
ZIP_DIR_NAME

'johnsmith_mn_test'

<p>
    This set of code is used to for troubleshoot sets by seeing if there is active connection to the Google server and you are receiving responses from Google
</p>

In [82]:
# This is troubleshooting code to see if the server is connected and you are receiving responses from Google.

import smtplib

smtp_server = "smtp.gmail.com"
smtp_port = 587

server = smtplib.SMTP(smtp_server, smtp_port)
server.set_debuglevel(1)  # Enables debugging output
server.starttls()
server.ehlo()  # Optional but ensures connection
print("SMTP server connection successful")

server.set_debuglevel(1)



SMTP server connection successful


send: 'ehlo [10.0.0.56]\r\n'
reply: b'250-smtp.gmail.com at your service, [98.52.208.169]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [98.52.208.169]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [10.0.0.56]\r\n'
reply: b'250-smtp.gmail.com at your service, [98.52.208.169]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your servic

<p>
    The following code will work if you have a Google workspace domain. This is does not work specifically with you are using an @gmail.com address due to Googles additional security steps for their free email service. 
</p>

In [ ]:
# This code is the odd way of creating a Python code to send emails using GMAIL. Note I have felt the Variable "Password" undefined.
#  This code does not work anymore because google have changed its security features.

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# Function to send an email with an attachment
def send_email(sender_email, password, recipient_email, subject, body, attachment_path=None):
    try:
        # Create the email object
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject

        # Attach the body text
        msg.attach(MIMEText(body, 'plain'))

        # Attach the file if provided
        if attachment_path:
            with open(attachment_path, 'rb') as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename={attachment_path.split("/")[-1]}'
            )
            msg.attach(part)

        # Set up the SMTP server
        smtp_server = "smtp.gmail.com"
        smtp_port = 587
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.set_debuglevel(1)  # Debugging enabled
        server.starttls()

        # Authenticate with Gmail's SMTP server
        server.login(sender_email, password)

        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())
        server.quit()

        print(f"Email sent successfully to {recipient_email}")
    except smtplib.SMTPAuthenticationError:
        print("Authentication failed. Please check your email and app password.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage (replace with actual sender, recipient, and file paths for testing)
send_email(
    sender_email="Steve.Jackson.1980.01@gmail.com",
    # Note, this is where you would input the user password.
    password=password,
    recipient_email="jBUCSA4@gmail.com",
    subject="Test Email with Attachment",
    body="This is a test email with an attachment.",
    attachment_path=None  # Replace with a valid file path if needed
)


<p>
    This code acts as a type of work around from by a Google Domain. This code requires the user to create a Developer Account, and use an API key with a token in the form of a .JSON file.
</p>

In [110]:

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os
from googleapiclient.discovery import build # type: ignore
import base64


# Function to send an email with an attachment using OAuth2
def send_email(sender_email, recipient_email, subject, body, attachment_path):
    try:
        # Set up the OAuth2 credentials
        SCOPES = ['https://www.googleapis.com/auth/gmail.send']
        creds = None

        # Load credentials from token.json if available
        if os.path.exists('../../Monick-Assessment/Dev_Test/token.json'):
            creds = Credentials.from_authorized_user_file('../../Monick-Assessment/Dev_Test/credentials.json', SCOPES)
        print("Loading credentials...")

        # If no valid credentials are available, log in and save the credentials
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('../../Monick-Assessment/Dev_Test/credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)

            # Save the credentials for future use
            with open('../../Monick-Assessment/Dev_Test/token.json', 'w') as token:
                token.write(creds.to_json())

        # Create the email object
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        print("Creating the email object...")
        # Attach the body text
        msg.attach(MIMEText(body, 'plain'))

        # Attach the file if provided
        if attachment_path:
            with open(attachment_path, 'rb') as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename={attachment_path.split('/')[-1]}'
            )
            msg.attach(part)

        # Send the email using Gmail's API
        
        print("Sending the email...")
        service = build('gmail', 'v1', credentials=creds)
        raw_message = base64.urlsafe_b64encode(msg.as_string().encode('utf-8')).decode('utf-8')
        message = {'raw': raw_message}
        service.users().messages().send(userId='me', body=message).execute()

        print(f"Email sent successfully to {recipient_email}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage (replace with actual sender, recipient, and file paths for testing)
send_email(
    sender_email="Steve.Jackson.1980.01@gmail.com",
    recipient_email="jbucsa4@gmail.com",
    subject="Test Email with Attachment",
    body="This is a test email with an attachment.",
    attachment_path=None  # Replace with a valid file path if needed
)


An error occurred: Authorized user info was not in the expected format, missing fields client_secret, refresh_token, client_id.


<h1>7) IQFeed Symbol Lookup

This task requires no code. Please follow the instructions:

- go to www.iqfeed.net
- go to their symbol lookup search engine
- find the "continuous contract code" for these 5 products:

1. Crude Oil future
2. S&P 500 future
3. Natural Gas Future
4. Eurostoxx 50 future
5. Brent Crude future


In [12]:
iqf_crude_oil = "QCL#"
iqf_s_and_p_500 = "@ES#"
iqf_nat_gas = "QNG#"
iqf_eurostoxx_50 = "EX#"
iqf_brent_crude = "EB#"